
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman;">
    TIME SERIES DATA PROCESSING | NET TRANSFER CAPACITIES
</div>
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman;">
    Downloading / Formatting Notebook
<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Each part of the following script is used to donwload and formating the raw data for the Net Transfer Capacities Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    1. Notebook Set Up
</div>

<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
    Importing needed libraries
</div>

In [17]:
import pandas as pd
from entsoe import EntsoePandasClient
from entsoe.exceptions import NoMatchingDataError  # Ensure this line is included
import os
import csv
from datetime import datetime
import pytz
from pytz import timezone, utc
import shutil

 <div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    2. ENTSO-E RESTful API.
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Connecting with the ENTSO-E API Tool.
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- To donwload the neeeded data using the API tool, is mandatory to use a token autentication to connect and make the future request of data.
</div>

In [18]:
client = EntsoePandasClient(api_key='61e5bbbb-7e80-4540-a471-bd993873aa74')

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    3. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [19]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    4. Usefull Variable Definition
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- Indicate the year of all data is referring to in the variable data_year.
<br>
- The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
<br>
- Additionally there are some default parameters that has to be defined to the correct working and calling to the ENTSO-E downloading functions.
</div>

In [20]:
# Year to which data refers to:
data_year = 2023
data_year_1 = data_year + 1

In [21]:
# Additional string to be appended
additional_path = "/RawData/NTC/"
additional_path_1 = "/RawData/NTC/Raw_Data_Sources/"

# Construct the Outage_Factors_folder_path variable
ntc_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Outage_Factors_Raw_Data_folder_path variable
ntc_raw_data_folder_path = dispaSET_unleash_folder_path + additional_path_1

print("ntc_folder_path:", ntc_folder_path)
print("ntc_raw_data_folder_path:", ntc_raw_data_folder_path)

ntc_folder_path: /home/ray/Dispa-SET_Unleash/RawData/NTC/
ntc_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/


In [22]:
# Define other parameters
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A51'

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    5. Country List Variable Definition
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Defining the list of countries according to the available data. 
</div>

In [23]:
# Define a list of country codes
ntc_per_unit_country_list = ["AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "GR", "ES", "FI", "FR", "HR", "HU", 
                             "IE", "IT", "LT", "LU", "LV", "MT", "NL", "NO", "PL", "PT", "RO", "SE", "SI", "SK", "UK"]

In [24]:
# Define the directory and file path
file_name = 'country_list.csv'
file_path = os.path.join(ntc_raw_data_folder_path, file_name)

# Ensure the directory exists
os.makedirs(ntc_raw_data_folder_path, exist_ok=True)

# Create a DataFrame
df = pd.DataFrame(ntc_per_unit_country_list, columns=['Country_From'])

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print(f"DataFrame saved to '{file_path}'")
ntc_country_list_file = file_path

DataFrame saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/country_list.csv'


<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [25]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"ntc_folder_path:                                           {ntc_folder_path}")   
print (f"ntc_raw_data_folder_path:                                  {ntc_raw_data_folder_path}")
print (f"ntc_country_list_file:                                     {ntc_country_list_file}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2023
ntc_folder_path:                                           /home/ray/Dispa-SET_Unleash/RawData/NTC/
ntc_raw_data_folder_path:                                  /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/
ntc_country_list_file:                                     /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/country_list.csv


<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Defining the sub-folders where all the cross border flows raw data is saved. 
</div>

In [26]:
# Convert data_year to string if it's not already
data_year = str(data_year)

# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Ensure the column 'Country_From' exists
if 'Country_From' not in df.columns:
    raise ValueError("Column 'Country_From' does not exist in the CSV file")

# Define the base directory where subfolders will be created
base_directory = os.path.join(ntc_raw_data_folder_path, data_year)

# Create a list to hold the paths of the created subfolders
country_folder_paths = []

# Create subfolders and save their paths
for country in df['Country_From']:
    # Create the subfolder path
    subfolder_path = os.path.join(base_directory, country)
    
    # Create the subfolder if it doesn't exist
    os.makedirs(subfolder_path, exist_ok=True)
    
    # Append the subfolder path to the list
    country_folder_paths.append(subfolder_path)

# Add the new column 'Country_Folder' to the DataFrame
df['Country_Folder'] = country_folder_paths

# Save the updated DataFrame back to the CSV file
df.to_csv(ntc_country_list_file, index=False)

print(f"Updated CSV file saved to '{ntc_country_list_file}' with new subfolders created.")

Updated CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/country_list.csv' with new subfolders created.


<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Defining the neighbor countries. 
</div>

In [27]:
# Raw data as a multiline string
data = """
CZ, DE, HU, IT, SI, CH
FR, DE, LU, NL, UK
GR, RO, 
AT, FR, DE, IT

AT, DE, PL, SK
AT, BE, CZ, DK, FR, LU, NL, NO, PL, SE, CH
DE, NL, NO, SE, UK
FI, LV
BG, IT
FR, PT
EE, NO, SE
BE, DE, IT, ES, CH, UK
HU, SI
AT, HR, RO, SK, SI
UK
AT, FR, GR, MT, SI, CH
LV, PL, SE
BE, DE
EE, LT
IT
BE, DK, DE, NO, UK
DK, FI, DE, NL, SE, UK
CZ, DE, LT, SK, SE
ES
BG, HU
DK, FI, DE, LT, NO, PL
AT, HR, HU, IT
CZ, HU, PL
BE, DK, FR, IE, NL, NO
"""

# Split the data into lines
lines = data.strip().split("\n")

# Initialize a list to hold the data
data_list = []

# Process each line
for line in lines:
    if line.strip() == "":
        # If the line is empty, add 11 empty strings
        data_list.append([""] * 11)
    else:
        neighbors = line.split(", ")
        data_list.append(neighbors)

# Create the DataFrame for new data
new_df = pd.DataFrame(data_list, columns=[f"Neighbor_{i}" for i in range(1, 12)])

# Path to the existing CSV file
existing_csv_file_path = ntc_country_list_file

# Read the existing CSV file into a DataFrame
existing_df = pd.read_csv(existing_csv_file_path)

# Concatenate the existing DataFrame with the new DataFrame horizontally
combined_df = pd.concat([existing_df, new_df], axis=1)

# Save the combined DataFrame back to the CSV file
combined_df.to_csv(existing_csv_file_path, index=False)
print(f"Data appended to '{existing_csv_file_path}' as new columns")

Data appended to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/country_list.csv' as new columns


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    6. Raw Data Download
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Donwloading the cross border flows raw data. 
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- The crows border flow data is downloaded in separate files for each country.
<br>
- Since the Acronym of Grece in the downloaded data is 'GR' and the Dispa-SET format for the country is 'EL'. All the needed changes in the used variables are done.
<br>
- The downloaded files will be joined into a single csv file under the name of the country which the flow comes from.
<br>
- The headers of these joined csv files are changed accordign the Dispa-SET cross border flow data format e.g. BE -> DE
</div>

In [28]:
# Define the start and end timestamps using the data_year variable
start = pd.Timestamp(f'{data_year}0101', tz='Europe/Brussels')
end = pd.Timestamp(f'{data_year_1}0101', tz='Europe/Brussels')

# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Ensure the necessary columns exist
required_columns = ['Country_From', 'Country_Folder'] + [f'Neighbor_{i}' for i in range(1, 12)]
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' does not exist in the CSV file")

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    country_code_from = row['Country_From']
    country_folder = row['Country_Folder']
    
    # Create the directory if it does not exist
    os.makedirs(country_folder, exist_ok=True)
    
    # Iterate through each neighbor column
    for neighbor_col in [f'Neighbor_{i}' for i in range(1, 12)]:
        country_code_to = row[neighbor_col]
        
        # Skip if the neighbor field is empty
        if pd.isna(country_code_to):
            continue
        
        try:
            # Query crossborder flows
            net_transfer = client.query_crossborder_flows(
                country_code_from, country_code_to, start=start, end=end
            )
            
            # Convert the index to a column
            net_transfer = net_transfer.reset_index()
            
            # Define the output file path
            output_file = os.path.join(country_folder, f'{country_code_to}.csv')
            
            # Save the DataFrame to a CSV file, including the index as a column
            net_transfer.to_csv(output_file, index=False)

            print(f"Data for {country_code_from} to {country_code_to} saved to '{output_file}'")
        
        except NoMatchingDataError:
            print(f"No matching data for {country_code_from} to {country_code_to} for the period {start} to {end}. Skipping.")

print("All data has been processed and saved.")

Data for AT to CZ saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/CZ.csv'
Data for AT to DE saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/DE.csv'
Data for AT to HU saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/HU.csv'
Data for AT to IT saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/IT.csv'
Data for AT to SI saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/SI.csv'
Data for AT to CH saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/CH.csv'
Data for BE to FR saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BE/FR.csv'
Data for BE to DE saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BE/DE.csv'
Data for BE to LU saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BE/LU.csv'
Data for BE to NL saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/B

Connection Error, retrying in 0 seconds


Data for FI to EE saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/FI/EE.csv'


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Replace 'GR' with 'EL' in the entire DataFrame
df = df.applymap(lambda x: x.replace('GR', 'EL') if isinstance(x, str) else x)

# Save the updated DataFrame back to the CSV file
df.to_csv(ntc_country_list_file, index=False)

print(f"Replacements made and file saved: {ntc_country_list_file}")

In [ ]:
# Walk through the directory tree
for root, dirs, files in os.walk(ntc_raw_data_folder_path, topdown=False):
    # Rename files
    for name in files:
        if 'GR' in name:
            new_name = name.replace('GR', 'EL')
            old_file_path = os.path.join(root, name)
            new_file_path = os.path.join(root, new_name)
            os.rename(old_file_path, new_file_path)
            print(f"Renamed file: {old_file_path} to {new_file_path}")

    # Rename directories
    for name in dirs:
        if 'GR' in name:
            new_name = name.replace('GR', 'EL')
            old_dir_path = os.path.join(root, name)
            new_dir_path = os.path.join(root, new_name)
            os.rename(old_dir_path, new_dir_path)
            print(f"Renamed directory: {old_dir_path} to {new_dir_path}")

In [ ]:
# Function to replace 'GR' with 'EL'
def replace_gr_with_el(lst):
    return ['EL' if x == 'GR' else x for x in lst]

# Applying the function to both lists
data = replace_gr_with_el(data)
ntc_per_unit_country_list = replace_gr_with_el(ntc_per_unit_country_list)

# Print the updated lists
print("Updated data list:", data)
print("Updated ntc_per_unit_country_list:", ntc_per_unit_country_list)

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Ensure the column 'Country_Folder' exists
if 'Country_Folder' not in df.columns:
    raise ValueError("Column 'Country_Folder' does not exist in the CSV file")

# Function to join CSV files in a directory
def join_csv_files_in_directory(directory_path):
    csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    if not csv_files:
        return None
    
    # Read all CSV files into DataFrames
    dataframes = {csv_file: pd.read_csv(os.path.join(directory_path, csv_file)) for csv_file in csv_files}
    
    # Find the CSV file with the largest number of rows
    largest_file = max(dataframes, key=lambda x: len(dataframes[x]))
    base_df = dataframes[largest_file].iloc[:, :2].copy()
    base_df.columns = [base_df.columns[0], largest_file.replace('.csv', '')]
    
    # Merge the other CSV files based on the first column
    for csv_file, df in dataframes.items():
        if csv_file == largest_file:
            continue
        temp_df = df.iloc[:, [0, 1]]
        temp_df.columns = [temp_df.columns[0], csv_file.replace('.csv', '')]
        base_df = pd.merge(base_df, temp_df, on=base_df.columns[0], how='left')
    
    return base_df

# Create a new column for the paths of the new CSV files
df['Country_File_Path'] = ''

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    country_folder = row['Country_Folder']
    
    # Join CSV files in the directory
    joined_df = join_csv_files_in_directory(country_folder)
    
    if joined_df is not None:
        # Define the output file path
        output_file = os.path.join(country_folder, f"{os.path.basename(country_folder)}.csv")
        
        # Save the joined DataFrame to a new CSV file
        joined_df.to_csv(output_file, index=False)
        
        # Update the DataFrame with the path of the new CSV file
        df.at[index, 'Country_File_Path'] = output_file

        print(f"Joined CSV file saved to '{output_file}'")

# Save the updated DataFrame back to the main CSV file
df.to_csv(ntc_country_list_file, index=False)

print("All data has been processed and saved.")

In [ ]:
# Read the main CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Ensure the required columns exist
if 'Country_From' not in df.columns or 'Country_File_Path' not in df.columns:
    raise ValueError("The CSV file must contain 'Country_From' and 'Country_File_Path' columns.")

# Function to update the headers of a CSV file
def update_csv_headers(file_path, new_header_prefix):
    # Read the CSV file into a DataFrame
    csv_df = pd.read_csv(file_path)
    
    # Get the current headers
    current_headers = csv_df.columns.tolist()
    
    # Create new headers for columns from the second column onward
    new_headers = [current_headers[0]] + [f"{new_header_prefix} -> {col}" for col in current_headers[1:]]
    
    # Update the DataFrame with the new headers
    csv_df.columns = new_headers
    
    # Save the updated DataFrame back to the CSV file
    csv_df.to_csv(file_path, index=False)
    print(f"Updated headers in '{file_path}'")

# Iterate through each row in the main DataFrame
for index, row in df.iterrows():
    country_from = row['Country_From']
    country_file_path = row['Country_File_Path']
    
    # Check if the file path is not empty and exists
    if pd.notna(country_file_path) and os.path.exists(country_file_path):
        update_csv_headers(country_file_path, country_from)
    else:
        print(f"File path '{country_file_path}' does not exist or is empty. Skipping...")

print("All CSV files have been processed.")

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    7. Raw Data Format
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Addapting the time step data to the UTC for all the countries.
</div>

In [ ]:
# Read the country list CSV file
country_list_df = pd.read_csv(ntc_country_list_file)

# Ensure the 'Country_File_Path' column exists
if 'Country_File_Path' not in country_list_df.columns:
    raise ValueError("Column 'Country_File_Path' does not exist in the CSV file")

# Define the function to convert time to UTC
def convert_to_utc(time_str):
    local_time = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S%z')
    utc_time = local_time.astimezone(pytz.utc)
    return utc_time.strftime('%Y-%m-%d %H:%M:%S%z')

# Process each CSV file
for file_path in country_list_df['Country_File_Path'].dropna():
    # Ensure the file exists
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        continue

    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Check if the 'index' column exists
    if 'index' not in df.columns:
        print(f"'index' column not found in file: {file_path}")
        continue

    # Convert the 'index' column to UTC
    df['index'] = df['index'].apply(convert_to_utc)
    
    # Save the updated CSV file
    df.to_csv(file_path, index=False)
    print(f"Updated file saved: {file_path}")

In [ ]:
# Read the country list CSV file
country_list_df = pd.read_csv(ntc_country_list_file)

# Ensure the 'Country_File_Path' column exists
if 'Country_File_Path' not in country_list_df.columns:
    raise ValueError("Column 'Country_File_Path' does not exist in the CSV file")

# Function to update the year in the 'index' column
def update_index_year(df, data_year):
    # Ensure the 'index' column exists
    if 'index' not in df.columns:
        raise ValueError("'index' column not found in DataFrame")
    
    # Update the year in the 'index' column
    df['index'] = df['index'].apply(lambda x: f"{data_year}{x[4:]}" if str(x)[:4] != str(data_year) else x)
    
    return df

# Process each CSV file specified in the 'Country_File_Path' column
for file_path in country_list_df['Country_File_Path'].dropna():
    # Ensure the file exists
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        continue
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Ensure there are enough rows to move the first four rows to the last
    if len(df) < 4:
        print(f"Not enough rows to process in file: {file_path}")
        continue
    
    # Extract the first four rows (excluding headers)
    first_four_rows = df.iloc[:4].copy()
    
    # Drop the first four rows from the DataFrame
    df = df.iloc[4:].reset_index(drop=True)
    
    # Append the first_four_rows to the end of the DataFrame
    df = pd.concat([df, first_four_rows]).reset_index(drop=True)
    
    # Update the 'index' column year
    df = update_index_year(df, data_year)
    
    # Save the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)
    print(f"Updated file saved: {file_path}")


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    7. Net Transfer Capacities Clean File
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Joining all the cros border flows data to a single csv file with named as the analized year.
</div>

In [ ]:
# Read the country list CSV file
country_list_df = pd.read_csv(ntc_country_list_file)

# Ensure the 'Country_File_Path' column exists
if 'Country_File_Path' not in country_list_df.columns:
    raise ValueError("Column 'Country_File_Path' does not exist in the CSV file")

# Process each CSV file specified in the 'Country_File_Path' column
file_paths = country_list_df['Country_File_Path'].dropna().tolist()

# Identify the CSV file with the largest number of rows
max_rows = 0
base_df = None
for file_path in file_paths:
    # Ensure the file exists
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        if len(df) > max_rows:
            max_rows = len(df)
            base_df = df.copy()

# If no base_df was found, raise an error
if base_df is None:
    raise ValueError("No valid CSV files found.")

# Initialize the combined DataFrame with the first column from the base DataFrame
combined_df = pd.DataFrame(base_df.iloc[:, 0])
combined_df.columns = [base_df.columns[0]]  # Keep the original name of the first column

# Add data from each CSV file to the combined DataFrame
for file_path in file_paths:
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        # Merge the data based on the first column
        combined_df = pd.merge(combined_df, df, on=base_df.columns[0], how='left')

# Save the combined DataFrame to a new CSV file named after the data_year variable
output_file_path = os.path.join(ntc_folder_path, f"{data_year}.csv")
combined_df.to_csv(output_file_path, index=False)
print(f"Combined CSV file saved: {output_file_path}")

<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Dividing the clean data in time stepts of 15 minutes, 30 minutes, and 1 hour.
</div>

In [ ]:
csv_file_path = os.path.join(ntc_folder_path, f'{data_year}.csv')

# Create the new directories
intervals = ['1h', '30min', '15min']
for interval in intervals:
    os.makedirs(os.path.join(ntc_folder_path, interval), exist_ok=True)

# Read the original CSV file
df = pd.read_csv(csv_file_path)

# Convert the 'index' column to datetime
df['index'] = pd.to_datetime(df['index'], format='%Y-%m-%d %H:%M:%S%z')

# Function to extract rows at a specific time step and save to a new CSV file
def extract_and_save(df, interval, folder_name):
    # Resample the DataFrame
    resampled_df = df.set_index('index').resample(interval).first().reset_index()
    
    # Define the new file path
    new_file_path = os.path.join(ntc_folder_path, folder_name, f'{data_year}.csv')
    
    # Save the resampled DataFrame to the new CSV file
    resampled_df.to_csv(new_file_path, index=False)
    print(f"File saved: {new_file_path}")

# Extract and save rows at different time steps
extract_and_save(df, '1H', '1h')
extract_and_save(df, '30T', '30min')
extract_and_save(df, '15T', '15min')

<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Copying the time already formated Net Transfer Capacities data to the main Dispa-SET data base dirtectory
</div>

In [ ]:
additional_path_2 = "/Database/NTC/"

# Construct the power_plants_raw_data_folder_path variable
ntc_data_base_folder_path = dispaSET_unleash_folder_path + additional_path_2

In [ ]:
# Define the subfolder names
subfolders = ['1h', '30min', '15min']

# Function to copy files
def copy_files(data_year, source_base_path, dest_base_path, subfolders):
    for subfolder in subfolders:
        source_path = os.path.join(source_base_path, subfolder, f"{data_year}.csv")
        dest_folder_path = os.path.join(dest_base_path, subfolder)

        # Create the destination subfolder if it does not exist
        os.makedirs(dest_folder_path, exist_ok=True)

        dest_path = os.path.join(dest_folder_path, f"{data_year}.csv")
        
        # Copy the file
        if os.path.isfile(source_path):
            shutil.copy2(source_path, dest_path)
            print(f"Copied {source_path} to {dest_path}")
        else:
            print(f"File {source_path} does not exist")

# Call the function
copy_files(data_year, ntc_folder_path, ntc_data_base_folder_path, subfolders)